# Exemples de categorització

In [1]:
# Exemple amb factorize(): Converteix variables categòriques en numèriques, sense importar-ne la relació numèrica.

import pandas as pd

data = pd.Series(['b', 'b', 'a', 'c', 'b'])
codificacio, indexUnics = pd.factorize(data)
print("Dades codificades: ",codificacio)
print("Codis/Índex utilitzats", indexUnics)

Dades codificades:  [0 0 1 2 0]
Codis/Índex utilitzats Index(['b', 'a', 'c'], dtype='object')


En aquest exemple, `codificacio` conté els valors numèrics associats a les categories, mentre que `indexUniques` conté els valors únics utilitzats per als índex: el 0 é la `b`, l'1 és la `a` i el 2 és la `c`.

La funció `factorize` admet un paràmetre *sort* que ordena prèviament les categoríes. Mirem-ho al següent exemple:

In [155]:
codificacio, indexUnics = pd.factorize(data,sort=True)
print("Dades codificades: ",codificacio)
print("Codis/Índex utilitzats", indexUnics)

Dades codificades:  [1 1 0 2 1]
Codis/Índex utilitzats Index(['a', 'b', 'c'], dtype='object')


En aquest cas, el 0 és la `a`, l'1 és la `b`i el 2 és la `c`.

Per altra banda, els valors nuls (`NaN`) es codifiquen com a -1. Per exemple:

In [156]:
codis, index=pd.factorize(pd.Series(['b', None, 'a', 'c', 'b']))

print("Dades codificades: ",codis)
print("Codis/Índex utilitzats", index)


Dades codificades:  [ 0 -1  1  2  0]
Codis/Índex utilitzats Index(['b', 'a', 'c'], dtype='object')


Aquest valor -1 es coneix com sentinella. Si en lloc de -1 volem que es tracten els nuls com un valor més, i per tant s'associe un índex, farem ús del paràmetre booleà `use_na_sentinel`. Aquest té com a valor predetarminat *True*, pel que fa ús del -1 com a sentinella. Si l'establim a *False*, es crearà un índex nou.

Veiem-ho amb un exemple:

In [157]:
import numpy as np

# Exemple amb use_na_sentinel a True (predeterminat)
codis,index = pd.factorize(pd.Series([1, 1, np.NaN, 2, 3, 3, 4, None]))

print("Dades codificades: ",codis)
print("Codis/Índex utilitzats", index)

# Exemple amb use_na_sentinel a False
codis,index = pd.factorize(pd.Series([1, 1, np.NaN, 2, 3, 3, 4, None]), use_na_sentinel=False)

print(" \nDades codificades: ",codis)
print("Codis/Índex utilitzats", index)

Dades codificades:  [ 0  0 -1  1  2  2  3 -1]
Codis/Índex utilitzats Index([1.0, 2.0, 3.0, 4.0], dtype='float64')
 
Dades codificades:  [0 0 1 2 3 3 4 1]
Codis/Índex utilitzats Index([1.0, nan, 2.0, 3.0, 4.0], dtype='float64')


Com veiem, al prime cas, els nuls (tant `np.NaN` com `None`) s'han codificat amb -1, i no s'ha creat un índex. En canvi, al segon exemple Veiem que s'ha creat un índex 1 per al valor `nan`, i els valors que eren nuls, ara prenen aquest valor 1.

### Exercici

**Proveu a factoritzar el següent Dataframe**

```py
df = pd.DataFrame([["VVLL 1","Rojo",1100], ["Seat","Verde",1200],
                   ["Audi",np.NaN,1230], ["Seat",np.NaN,1300],
                   [np.NaN,"Verde",1450], ["Citr A1","Blanco",1000],
                   ["Ford","Rojo",1500],["BBWW","Rojo",1500],
                   ["Dacia",np.NaN,1450], [np.NaN,"Blanco",1000],
                   ["Tesla","Rojo",3500],["HunDai",np.NaN,1500],
                   ], 
                  columns=["modelo", "color","precio"])
```

**Creeu una nova cel·la al notebook i intenteu respondre abans de veure la solució***

Pistes:

* Cal aplicar factorització columna a columna
* Per juntar les columnes de nou fem ús de la funció `stack` de numpy: `np.stack((columnes_separade_per_comes), axis=1)`

In [19]:
# Solució a l'exercici

# Importem les llibreries
import numpy as np
import pandas as pd

# Definim el DataFrame:

df = pd.DataFrame([["VVLL 1","Rojo",1100], ["Seat","Verde",1200],
                   ["Audi",np.nan,1230], ["Seat",np.nan,1300],
                   [np.nan,"Verde",1450], ["Citr A1","Blanco",1000],
                   ["Ford","Rojo",1500],["BBWW","Rojo",1500],
                   ["Dacia",np.nan,1450], [np.nan,"Blanco",1000],
                   ["Tesla","Rojo",3500],["HunDai",np.nan,1500],
                   ], 
                  columns=["modelo", "color","precio"])

print(df)




     modelo   color  precio
0    VVLL 1    Rojo    1100
1      Seat   Verde    1200
2      Audi     NaN    1230
3      Seat     NaN    1300
4       NaN   Verde    1450
5   Citr A1  Blanco    1000
6      Ford    Rojo    1500
7      BBWW    Rojo    1500
8     Dacia     NaN    1450
9       NaN  Blanco    1000
10    Tesla    Rojo    3500
11   HunDai     NaN    1500


In [21]:
import sklearn 
import preprocessing
df['modelo'].fillna('Deconocido', inplace=True)
df('color').fillna('Desconocido', inplace=True)
codiModelo, indexModel = pd.factorize(df['modelo'])
codiColor, indexColor = pd.factorize(df['color'])
precio_escalado = preprocessing.MinMaxScaler().fit_transform(df['precio'].values.reshape(-1,1))
datosMix = np.stack([codiModelo,codiColor, precio_escalado.reshape(-1)], axis=1)
dfdatosMix = pd.DataFrame(datosMix, columns=['modelo', 'color', 'precio'])
dfdatosMix

TypeError: 'DataFrame' object is not callable

## Categorització de dades categòriques ordinals

Anem a fer un exemple sobre les valoracions d'un restaurant. Als clients se'ls passa una enquesa sobre el servei i la qualitat dels productes, on les respostes estan organitzades en les següents categories:

* Satiscacció amb el servei:
    * Molt insatisfet
    * Insatisfet
    * Neutral
    * Satisfet
    * Molt satisfet

* Qualitat dels productes:
    * Mala,
    * Bona
    * Molt bona
    * Excel·lent


### Codificador OrinalEncoder

In [ ]:
import pandas as pd
import numpy as np

# Definim les diferents categories
categories_servei = ["Molt insatisfet", "Insatisfet", "Neutral","Satisfet", "Molt satisfet"]
categories_qualitat = ["Mala", "Bona", "Molt bona", "Excel·lent"]


# I ara definim les dades de l'enquesta realitzada

enquesta={"servei":["Molt insatisfet", "Insatisfet", "Neutral","Satisfet", "Molt satisfet","Molt insatisfet"],
           "qualitat":["Mala", "Bona", "Molt bona", "Excel·lent","Mala", "Bona"]}

# I definim el tipus de client de cada votació, sent:
#     0: client esporàdic,
#     1: client habitual

tipus_client=[0,0,1,1,0,1]

# Creem el dataframe de l'enquesta
pd.DataFrame(enquesta)

,servei,qualitat
0,Molt insatisfet,Mala
1,Insatisfet,Bona
2,Neutral,Molt bona
3,Satisfet,Excel·lent
4,Molt satisfet,Mala
5,Molt insatisfet,Bona


Anem a aplicar ara la codificació `OrdinalEncoder`, i observem-ne el resultat:

In [160]:
# Importem el codificador
from sklearn.preprocessing import OrdinalEncoder

# Creem el DataFrame per treballar amb ell
dades_ordinals=pd.DataFrame(enquesta)

# Incorporem les categories al codificador, sent el 0 la valoració més negativa
codificador=OrdinalEncoder(categories=[categories_servei, categories_qualitat ])

# Una vegada definit el codificador, li proporcionem les dades
# per ajustar-lo (fit) i transformar-lo (transform) (o tot d'una amb fit_transform)

# Això ens generarà un vector amb les categories transformades
codificador.fit(dades_ordinals)
dades=codificador.transform(dades_ordinals)

print(dades)

# I ara creem el Dataframe
dades_ordinals=pd.DataFrame(dades,columns=["servei","qualitat"]) 

dades_ordinals

[[0. 0.]
 [1. 1.]
 [2. 2.]
 [3. 3.]
 [4. 0.]
 [0. 1.]]


,servei,qualitat
0,0.0,0.0
1,1.0,1.0
2,2.0,2.0
3,3.0,3.0
4,4.0,0.0
5,0.0,1.0


Aquesta codificació és possible que no siga la millor, ja que assumeix que les categoríes són equidistans (és a dir, que la distància entre les diferents categories és la mateixa). Això podria afegir cert soroll en l'anàlisi i la creació del model. De tota manera és un codificador que ens permet establir cert ordre en cada categoria.

In [161]:
# Si volem veure els noms de les categories del codificador, podem fer ús de l'atribut categories_. 
# Podem utilitzar aquests noms si cal crear, per exemple un DataFrame.

print(codificador.categories_)

[array(['Molt insatisfet', 'Insatisfet', 'Neutral', 'Satisfet',
       'Molt satisfet'], dtype=object), array(['Mala', 'Bona', 'Molt bona', 'Excel·lent'], dtype=object)]


### Codificador One-Hot Encoder

El mètode OneHot Encoder no afig el soroll que puguen afegir les categoríes equidistants, però genera més columnes. 

Recordem que per cada possible valor es genera una columna, de manera que aquesta s'activa per al valor corresponent a la categoría.

In [162]:
# Importem les llibreries
from sklearn.preprocessing import OneHotEncoder

# Definim les dades
dades_one_hot=pd.DataFrame(enquesta)

# Declarem el codificador
codificador=OneHotEncoder()

# I ajustem els paràmetres (fit) i transformem tot d'Una
codificacio=codificador.fit_transform(dades_one_hot)

# Mostrem la matriu dispersa generada
print(codificacio)

print("Dimensions: ",codificacio.shape)

# I els noms de les categoríes del codificador
codificador.get_feature_names_out()

  (0, 1)	1.0
  (0, 7)	1.0
  (1, 0)	1.0
  (1, 5)	1.0
  (2, 3)	1.0
  (2, 8)	1.0
  (3, 4)	1.0
  (3, 6)	1.0
  (4, 2)	1.0
  (4, 7)	1.0
  (5, 1)	1.0
  (5, 5)	1.0
Dimensions:  (6, 9)


array(['servei_Insatisfet', 'servei_Molt insatisfet',
       'servei_Molt satisfet', 'servei_Neutral', 'servei_Satisfet',
       'qualitat_Bona', 'qualitat_Excel·lent', 'qualitat_Mala',
       'qualitat_Molt bona'], dtype=object)

Aquest codificador pot resultar una miqueta complex d'entendre. Hem comentat que el codificador genera una **matriu dispersa**. En àlgebra una matriu dispersa és aquella on la majoria dels elements són 0, i per tant, es pot representar d'una forma més simplificada. 

En aquest cas, el que ens mostra la codificació són aquelles posicions de la matriu on no hi ha un 0, i el valor que hi ha. Per exemple, en la posició (fila=0,columna=1) hi ha un 1, en la posició (fila=0, columna=7) hi ha un altre 1, i així fins la posició (fila 5, columna 5), on també hi ha un 1.

Una altra dada important a tindre en compte són les dimensions (6x9), on hi ha 6 files, corresponents a les 6 respostes que es té de l'enquesta i les 9 columnes resultants. Recordem que originalment teniem dues columnes amb dades categòriques. Fixeu-vos que el que s'ha fet ha estat que cadascuna d'aquestes columnes s'ha dividit en tantes columnes com valors podiem tindre. Així, la columna de servei, que tenia 5 categories, s'ha convertit en 5 columnes noves (`servei_Insatisfet`, `servei_molt insatisfet`, etc.) i la columna de qualitat, que tenia 4 categories s'ha convertit en quatre columnes més (`qualitat_Bona`, `qualitat_Excel·lent`, etc.).

Com podem apreciar, cada columna nova es nomena combinant el nom de la columna original (servei o qualitat), amb el nom de la categoria.

In [163]:

# Si volem vore la forma completa d'aquesta matriu podem convertir-la a un array de numpy:

print(codificacio.toarray()) # Mostrem la matriu dispsera al complet

[[0. 1. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0.]]


In [164]:
# I per vore les categoríes del codificador

print(codificador.categories_)

[array(['Insatisfet', 'Molt insatisfet', 'Molt satisfet', 'Neutral',
       'Satisfet'], dtype=object), array(['Bona', 'Excel·lent', 'Mala', 'Molt bona'], dtype=object)]


In [165]:
# Trambé podem veure els noms de les característiques resultants

print(codificador.get_feature_names_out())

['servei_Insatisfet' 'servei_Molt insatisfet' 'servei_Molt satisfet'
 'servei_Neutral' 'servei_Satisfet' 'qualitat_Bona' 'qualitat_Excel·lent'
 'qualitat_Mala' 'qualitat_Molt bona']


In [166]:
# Creem ara un dataframe amb les dades ja categoritzades
# Per a això fem ús de concatenate.
novesCol=np.concatenate(codificador.categories_)
# Creem el DataFrame amb les dades ja categoritzades i afegim el nom de les columnes
novesDades=pd.DataFrame(codificacio.toarray(), columns=novesCol)

novesDades

,Insatisfet,Molt insatisfet,Molt satisfet,Neutral,Satisfet,Bona,Excel·lent,Mala,Molt bona
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


Totes aquestes operacions també podrien haver-se fet conjuntament.



In [167]:
# Creem el dataframe amb tot junt:

from sklearn.preprocessing import OneHotEncoder

codificador=OneHotEncoder()

dades_one_hot_encoder = pd.DataFrame(codificador.fit_transform(dades_one_hot).toarray(),
                       columns=np.concatenate(codificador.categories_))
dades_one_hot_encoder

,Insatisfet,Molt insatisfet,Molt satisfet,Neutral,Satisfet,Bona,Excel·lent,Mala,Molt bona
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## L'eina CountVectorizer

`CountVectorizer` és una eina específica per al  preprocessament de text, que converteix una col·lecció de textos en una representació de vectors de comptadors de termes. Veiem el següent exemple:

In [168]:
from sklearn.feature_extraction.text import CountVectorizer

text = ["La Fuerza es mi aliado, y un poderoso aliado es. Vida crea, la hace crecer. \
         Su energía nos rodea y nos une. Somos seres luminosos", "Debes desaprender lo aprendido",
          "Cuando mires al lado oscuro, cuidado debes tener… ya que el lado Oscuro te mira también", 
          "Para vencer a un enemigo no es necesario matarlo. Derrota la rabia que hay en él y tu \
            enemigo no será más. La ira el verdadero enemigo es"]

# Definim el codificador
vectorizer = CountVectorizer()

# L'inicialitzem amb els textos
vectorizer.fit(text)

# I apliquem el transformador
vector = vectorizer.transform(text)

# Mostrem el resultat
print(vector.toarray())

# I veiem els índex amb get_feature_names_out
feature_names = vectorizer.get_feature_names_out()
print(feature_names)

[[0 2 0 1 1 0 0 0 0 0 0 0 0 1 2 1 1 0 0 2 0 0 1 0 1 0 0 0 0 0 2 0 0 1 0 0
  1 1 0 1 1 0 0 0 0 1 1 0 0 1 0 0]
 [0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 2 0 0 0 0 1 1 0 0 0 0 2 0 0 1 0
  0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0 1 1 3 0 2 0 0 1 1 2 0 0 0 1 0 0 0 1 1 2 0 0 1 0 1 1
  0 0 1 0 0 0 0 0 1 1 0 1 1 0 0 1]]
['al' 'aliado' 'aprendido' 'crea' 'crecer' 'cuando' 'cuidado' 'debes'
 'derrota' 'desaprender' 'el' 'en' 'enemigo' 'energía' 'es' 'fuerza'
 'hace' 'hay' 'ira' 'la' 'lado' 'lo' 'luminosos' 'matarlo' 'mi' 'mira'
 'mires' 'más' 'necesario' 'no' 'nos' 'oscuro' 'para' 'poderoso' 'que'
 'rabia' 'rodea' 'seres' 'será' 'somos' 'su' 'también' 'te' 'tener' 'tu'
 'un' 'une' 'vencer' 'verdadero' 'vida' 'ya' 'él']


# Comparant codificadors

Anem a fer ara una comparació d'ambdos codificadors, aprofitant que tenim la codificació feta, en un model d'aprenentatge. Així introduim també unes pinzellades sobre com treballem amb aquests. El model d'aprenentatge automàtic que utilitzarem serà un classificador molt bàsic: el classificador de **regressió logística**, que treballarem més a fons en unitats posteriors.

El probema que abordarem consistirà en determinar, segons les valoracions si qui ha fet la valoració és un client habitual o esporàdic. 

La **regressió logística** és un algorisme d'aprenentatge automàtic, i més concretament un *classificador binari*. Això significa que  la seua tasca és la classificació, i que la variable depenent (el que es prediu) és una variable binària categòrica. A més, es tracta d'**aprenentatge supervisat**, el que significa que per a l'entrenament li proporcionarem tant el conjunt de dades com el resultat esperat.

En aquest cas, disposem del conjunt de dades, o d'entrenament, que seran les valoracions realitzades, i per altra banda disposàvem també d'un vector que ens indicava si eren clients habituals o no.

El que anem a fer, en primer lloc serà escalar les dades obtingudes amb la codificació ordinal, i entrenarem l'algorisme amb aquestes.

Posteriorment, farem l'entrenament amb les dades codificades amb one-hot-encoding, aquesta vegada sense escalar, ja que és una matiu binària, i tornarem a fer l'entrenament.

In [169]:
# Escalem les dades generades amb el codificador

from sklearn.linear_model import LogisticRegression # Importem el model de Regressió Logística
from sklearn.preprocessing import MinMaxScaler # Importem el MinMaxScaker

print("\nDades codificades amb codificació ordinal")
escalador=MinMaxScaler()
print(dades_ordinals)

print("\nDades escalades amb codificació ordinal")
dades_ordinals=escalador.fit_transform(dades_ordinals)
print(dades_ordinals)


Dades codificades amb codificació ordinal
   servei  qualitat
0     0.0       0.0
1     1.0       1.0
2     2.0       2.0
3     3.0       3.0
4     4.0       0.0
5     0.0       1.0

Dades escalades amb codificació ordinal
[[0.         0.        ]
 [0.25       0.33333333]
 [0.5        0.66666667]
 [0.75       1.        ]
 [1.         0.        ]
 [0.         0.33333333]]


In [170]:
# Recordem que teniem definits els tipus de clients prèviament, 
# on 0 era un client esporàdic i 1 un client freqüent.

tipus_client

[0, 0, 1, 1, 0, 1]

In [171]:
print("\n Classificació amb dades codificades amb OrdinalEncoder")

# Entrenem el model amb fit. Aquest necessita dos paràmetres:
# * El primer són les dades d'entrenament, i pot ser qualsevol tipus de matriu, i 
# * El segon és el vector de targets, amb les eixides corresponents a la matriu.
# Al nostre cas, el vector d'entrenament són les dades ordinals, i els targets el tipus de client.

model=LogisticRegression().fit(dades_ordinals, tipus_client)

# Una vegada entrenat, fem la predicció sobe les mateixes dades d'entrenament
print("\n    Prediccions: ", model.predict(dades_ordinals))
print("\nEixida esperada: ", tipus_client)

# Veiem la probabilitat de que cada instància pertanga a cadascuna de les classes (client esporàdic / client freqüent)
print("\nProbabilitat de ser client esporàdic o client freqüent")
print("Esporàdic  |   Freqüent")
print("-----------------------")
print(model.predict_proba(dades_ordinals))


 Classificació amb dades codificades amb OrdinalEncoder

    Prediccions:  [0 0 1 1 0 0]

Eixida esperada:  [0, 0, 1, 1, 0, 1]

Probabilitat de ser client esporàdic o client freqüent
Esporàdic  |   Freqüent
-----------------------
[[0.56489598 0.43510402]
 [0.50845928 0.49154072]
 [0.45180614 0.54819386]
 [0.39637512 0.60362488]
 [0.57175647 0.42824353]
 [0.50671189 0.49328811]]


In [172]:
# Fem la predicció sobre les dades codificades amb One Hot Encoder
# Per a això no cal fer escalat previ, ja que es tracta d'una matriu dispersa amb 0s i 1s

print("\Classificació de les dades codificades amb OneHotEncoder")

model=LogisticRegression().fit(dades_one_hot_encoder, tipus_client)

# Predicció sobre les mateixes dades, una vegada entrenat

print("\nPrediccions: ", model.predict(dades_one_hot_encoder))
print("\nDades Esperades: ", tipus_client)

# Veiem la probabilitat de que cada instància pertanga a cadascuna de les classes (client esporàdic / client freqüent)
print("\nProbabilitat de ser client esporàdic o client freqüent")
print("Esporàdic  |   Freqüent")
print("-----------------------")
print(model.predict_proba(dades_one_hot_encoder))



\Classificació de les dades codificades amb OneHotEncoder

Prediccions:  [0 0 1 1 0 1]

Dades Esperades:  [0, 0, 1, 1, 0, 1]

Probabilitat de ser client esporàdic o client freqüent
Esporàdic  |   Freqüent
-----------------------
[[0.62301432 0.37698568]
 [0.57519021 0.42480979]
 [0.32279763 0.67720237]
 [0.32279763 0.67720237]
 [0.70505958 0.29494042]
 [0.45116842 0.54883158]]


Si observem la primera predicció realitzada, amb la codificació OrdinalEncoder ha encertat tots els resultats, llevat de l'últim, i amb la codificació OneHotEncoder els ha encertat tots.

**Com podem veure, el mateix algorisme d'aprenentatge automàtic (en aquest cas una regressió logística), entrenat de la mateixa manera i amb les mateixes dades d'origen, ha donat diferents resultats segons com hajam codificat aquestes. Veiem aci la importància d'una correcta adequació de les dades i un bon preprocessament abans d'abordar l'entrenament en sí.**

## Categòrics nominals

Anem a veure un exemple amb dades categòriques. En aquest cas, anem a treballar amb un DataFrame amb dades de persones, com el nom, el salari i el país. Aquesta última columna anem a convertir-la en categòrica.

In [4]:
import pandas as pd

# Definim les dades en un diccionari
dades = {"nom" : ["Mariana", "Ana", "Elsa", "Gustavo",
                     "Pedro", "Raúl", "Carlos", "José", "Luis"],
         
         "sou" : [10000.00, 8000.00, 9000.00, 2000.00,
                    2100.00, 12000.00, 5000.00, 10000.00, 200.00],
         
         "pais" : ["Argentina", "Bolivia", "Chile", "Colombia",
                   "Costa Rica", "Ecuador", "México", "Perú", "Perú"]}

# I el convertim a DataFrame

dades = pd.DataFrame(dades)
dades.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   nom     9 non-null      object 
 1   sou     9 non-null      float64
 2   pais    9 non-null      object 
dtypes: float64(1), object(2)
memory usage: 348.0+ bytes


És important observar que la columna `pais` (igual que `nom`) és de tipus `object`. Aquesta dada anem a convertir-la a categòrica amb el mètode `astype("catecogy")`, per canviar el tipus de columna a categòric, el que ens permetrà un processament més eficient.

In [5]:
dades["pais"]=dades["pais"].astype("category")

dades = pd.DataFrame(dades)
dades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   nom     9 non-null      object  
 1   sou     9 non-null      float64 
 2   pais    9 non-null      category
dtypes: category(1), float64(1), object(1)
memory usage: 649.0+ bytes


Com podem veure, la columna `pais`, en lloc de tindre el tipus `object` ara té un tipus `categoty.`

Per tal de codificar aquestes categoríes, un dels mètodes més utilitzats és OneHotEncoder. Veiem com aplicar-ho.

In [6]:
# Importem la classe
from sklearn.preprocessing import OneHotEncoder

# I definim un codificador de tipus One Hot
codificador=OneHotEncoder()

# Per tal de fer l'entrenament, i com que treballem amb
# DataFrames en lloc de series cal fer ús de la notació [[]]

# Indiquem que  aplique la transformació sobre la columna categòrica
codificacio=codificador.fit_transform(dades[["pais"]])

# I veiem el resultat
print(type(codificacio))
print(codificacio)
print(codificacio.toarray())

<class 'scipy.sparse._csr.csr_matrix'>
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9 stored elements and shape (9, 8)>
  Coords	Values
  (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	1.0
  (4, 4)	1.0
  (5, 5)	1.0
  (6, 6)	1.0
  (7, 7)	1.0
  (8, 7)	1.0
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


Com podem veure, i com era d'esperar, el resultat d'aquesta codificació ens genera una matriu dispera, on cada valor diferent s'ha convertit en una columna, on s'indica amb un 1 si el valor es correspon al de la columna i 0 en cas contrari. Podem veure aquesta cosificació amb el següent:

In [7]:
# Mostrem les categories
print(codificador.categories_)

# Creem ara una nova variable amb les columnes generades
noves_columnes=codificador.categories_

[array(['Argentina', 'Bolivia', 'Chile', 'Colombia', 'Costa Rica',
       'Ecuador', 'México', 'Perú'], dtype=object)]


In [14]:
# I constrium les noves columnes, amb el vector de numpy i els noms de columnes codificats en el codificador:

noves_columnes_categories=pd.DataFrame(codificacio.toarray(), columns=noves_columnes)

noves_columnes_categories

,Argentina,Bolivia,Chile,Colombia,Costa Rica,Ecuador,México,Perú
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
# I finalment, afegirm aquestes al DataFrame, mitjançant l'ordre pd.concat()

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

sou_escalat = preprocessing.MinMaxScaler().fit_transform(dades['sou'].values.reshape(-1,1))
codificador= OneHotEncoder()
dades_one_hot = pd.DataFrame(codificador.fit.transform(dades[['pais']]).toarray(), columns=np.concatenate(codificador.categories_))
sou_escalat_df= pd.DataFrame(sou_escalat, columns=['sou_escalat'])
dades = pd.concat([dades_one_hot, sou_escalat_df], axis=1)
dades_one_hot_encoder

AttributeError: 'function' object has no attribute 'transform'

Amb això, i com que estem representant el país en diverses columnes, no hi ha distinció ni infomració errònia que puga introduis biaixos en la mostra.

### Aplicant múltiples transformacions

Quan tenim característiques amb diferents tipus i volem aplicar diferents codificadors, podem fer ús de transformadors.

La classe `ColumnTransformer` d'Scikit-learn ajuda a realitzar diverses transformació p a diferents columnes de dades. Aquesta clase funciona tanb amb matrius com amb matrius disperses i DtaFrames de Pandas.

In [179]:
# Exemple: Tenim el següent DataFrame, amb dades heterogènies:

import pandas as pd

X = pd.DataFrame(
{ 'ciudad': ['London', 'London', 'Paris', 'Sallisaw'],
  'titulo': ["His Last Bow", "How Watson Learned the Trick",
           "A Moveable Feast", "The Grapes of Wrath"],
   'expert_rating': [5, 3, 4, 5],
   'user_rating': [4, 5, 4, 3]})
X

,ciudad,titulo,expert_rating,user_rating
0,London,His Last Bow,5,4
1,London,How Watson Learned the Trick,3,5
2,Paris,A Moveable Feast,4,4
3,Sallisaw,The Grapes of Wrath,5,3


Podriem per exemple voler codificar la *ciutat* com una variable categòrica, fent ús de *OneHotEncoder* però aplicant un *CountVectorizer* al títol. Com que podem fer ús de diversos mètodes d'extracció de característiques en la mateixa columna, podem donar a cada transformador un nom únic (p. ex. 'cat_ciutat' i 'titol_nou'). Recordem que de manera predeterminada, les columnes de qualificació restants s'ignoren (`remainder='drop'`).

In [180]:
# Importem ColumnTransformer, CountVectorizer i OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder

# Definim el ColumnTransformer amb un parell de tuples amb els transformadors:
# * La primera (cat_ciutat) aplica una codificació OneHotEncoder sobre la columna *ciudad*
# * La segona (titol_nou) aplica un CountVectorizer sobre el títol.
# Fem ús de remainder=drop, per a que la resta de columnes s'eliminen
# I amb verbose_features_names_out a False mantenim els noms de les característiques/columnes amb prefix
# Amb true, el transformador posa el nom que hem escrit (cat_ciutat, titol_ nou...)

column_trans = ColumnTransformer(
    [('cat_ciutat' , OneHotEncoder(dtype='int'), ['ciudad']),
     ('titol_nou', CountVectorizer(), 'titulo')],
      remainder='drop', verbose_feature_names_out=False) # si remainder='passthrough' les columnes es mantenen.

column_trans.fit(X)


ColumnTransformer(transformers=[('cat_ciutat', OneHotEncoder(dtype='int'),
                                 ['ciudad']),
                                ('titol_nou', CountVectorizer(), 'titulo')],
                  verbose_feature_names_out=False)

In [181]:
# Veiem ara els noms de les columnes que es van a generar
column_trans.get_feature_names_out()

array(['ciudad_London', 'ciudad_Paris', 'ciudad_Sallisaw', 'bow', 'feast',
       'grapes', 'his', 'how', 'last', 'learned', 'moveable', 'of', 'the',
       'trick', 'watson', 'wrath'], dtype=object)

In [182]:
# Transformem les dades i mostrem el vector generat
column_trans.transform(X).toarray()

array([[1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]])

In [183]:
# Finalment creem un dataframe amb les dades transformades

novesCol=column_trans.get_feature_names_out()
dades=pd.DataFrame(column_trans.transform(X).toarray(), columns=novesCol)
dades

,ciudad_London,ciudad_Paris,ciudad_Sallisaw,bow,feast,grapes,his,how,last,learned,moveable,of,the,trick,watson,wrath
0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1,0,1,0,0,1,1,1,0
2,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0
3,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1


In [184]:
# Veiem ara què passa si deixem remainder='passthrough'

column_trans_2 = ColumnTransformer(
    [('cat_ciutat' , OneHotEncoder(dtype='int'), ['ciudad']),
     ('titol_nou', CountVectorizer(), 'titulo')],
      remainder='passthrough', verbose_feature_names_out=False)

column_trans_2.fit(X)

ColumnTransformer(remainder='passthrough',
                  transformers=[('cat_ciutat', OneHotEncoder(dtype='int'),
                                 ['ciudad']),
                                ('titol_nou', CountVectorizer(), 'titulo')],
                  verbose_feature_names_out=False)

In [185]:
# Veiem ara els noms de les columnes que es van a generar 
# (observeu que apareixen també expert_rating i user_rating)
column_trans_2.get_feature_names_out()



array(['ciudad_London', 'ciudad_Paris', 'ciudad_Sallisaw', 'bow', 'feast',
       'grapes', 'his', 'how', 'last', 'learned', 'moveable', 'of', 'the',
       'trick', 'watson', 'wrath', 'expert_rating', 'user_rating'],
      dtype=object)

In [186]:
# Transformem les dades i mostrem el vector generat
# En aquest cas, no hem de fer la conversió a array, ja 
# que el resultat de la transformació és ja un array (no una matriu dispersa).
column_trans_2.transform(X)

array([[1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 5, 4],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 3, 5],
       [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 4, 4],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 5, 3]])

In [187]:
# Veiem ara el dataframe amb les dades transformades

novesCol_2=column_trans_2.get_feature_names_out()
dades_2=pd.DataFrame(column_trans_2.transform(X), columns=novesCol_2)
dades_2

,ciudad_London,ciudad_Paris,ciudad_Sallisaw,bow,feast,grapes,his,how,last,learned,moveable,of,the,trick,watson,wrath,expert_rating,user_rating
0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,5,4
1,1,0,0,0,0,0,0,1,0,1,0,0,1,1,1,0,3,5
2,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,4,4
3,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1,5,3


In [188]:
# També podem indicar en el transformador que aplique una escala a totes les dades,
# per a que estiguen entre 0 i 1.

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder

column_trans = ColumnTransformer(
    [('cat_ciutat', OneHotEncoder(dtype='int'), ['ciudad']),
     ('nou_titol', CountVectorizer(), 'titulo')],
      remainder=MinMaxScaler(), verbose_feature_names_out=False)
column_trans.fit(X)

column_trans.get_feature_names_out()

dd=column_trans.transform(X)
print(dd)

novesCol=column_trans.get_feature_names_out()
dades=pd.DataFrame(column_trans.transform(X), columns=novesCol)
dades


[[1.  0.  0.  1.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.5]
 [1.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  1.  1.  1.  0.  0.  1. ]
 [0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.5 0.5]
 [0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  1.  1.  0.  0.  1.  1.  0. ]]


,ciudad_London,ciudad_Paris,ciudad_Sallisaw,bow,feast,grapes,his,how,last,learned,moveable,of,the,trick,watson,wrath,expert_rating,user_rating
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.5
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0


# Discretització de variables

Recordem que consisteix en la transformació d'una variable numèrica en categòrica, i que sol agrupar les dades en intervals.

La compertimentació es pot fer bé amb *contenidors homogenis* o amb *contenidors no homogenis*.

Com a exemple, anem a discretitzar el següent array d'edats:

```py
edats = np.array([1, 7, 8, 15, 16, 28, 35, 50, 55, 70, 75, 100])
```

## Contenidors homogenis

In [189]:
# Importem les lliberies
import numpy as np
import pandas as pd

# Definim el vector d'edats
edats = np.array([1, 7, 8, 15, 16, 28, 35, 50, 55, 70, 75, 100])

# Fem ús de cut per fer les particions

resultat=pd.cut(edats,   # Proporcionem la matriu unidimensional d'entrada
                bins=3,  # nombre de contenidors desitjats
                labels=["baixa", "mitja", "alta"], # Etiquetes, com que bins és 3, n'hem de posar 3
                include_lowest=True, # Per a que incloga el valor més baixet
                retbins=True # Indiquem que retorne els rangs utilitzats i la grandària dels contenidors
                )

# resultat és un vector de dos columnes, la primera les dades categoritzades, i la segona 
# les dades dels contenidors. Podem mostrar-ho amb:
print("Dades categoritzades: ", resultat[0],"\n")
print("Contenidors", resultat[1],"\n")

# Codis associats
print("Codis Associats\n")
print(resultat[0].codes, "\n")

# Categories
print( " Categories creades")
print(resultat[0].categories, "\n")

# Mostraem les dades convertides dins un array de numpy
print( "Dades convertides")
print(np.array(resultat[0]), "\n")


Dades categoritzades:  ['baixa', 'baixa', 'baixa', 'baixa', 'baixa', ..., 'mitja', 'mitja', 'alta', 'alta', 'alta']
Length: 12
Categories (3, object): ['baixa' < 'mitja' < 'alta'] 

Contenidors [  0.901  34.     67.    100.   ] 

Codis Associats

[0 0 0 0 0 0 1 1 1 2 2 2] 

 Categories creades
Index(['baixa', 'mitja', 'alta'], dtype='object') 

Dades convertides
['baixa' 'baixa' 'baixa' 'baixa' 'baixa' 'baixa' 'mitja' 'mitja' 'mitja'
 'alta' 'alta' 'alta'] 



In [190]:
dades=pd.cut(edats,    bins=3,      labels=["baixa",  "mitjana",  "alta"],  include_lowest=True)
pd.value_counts(dades)

/tmp/ipykernel_7905/276883711.py:2: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(dades)


baixa      6
mitjana    3
alta       3
Name: count, dtype: int64

## Contenidors no homogenis

In [191]:
edats  =  np.array([1,  7,  8,  15,  16,  28,  35,  50,  55,  70,  75,  100])
resultat  =  pd.cut(edats,
                    bins=[0,  11,  17,  59,  np.inf],  #  Aci indiquem els rangs de cada contenidor
                                                        # # np.inf representa al infinito positivo
                    labels=["xiquet",  "jove",  "adult",  "major"],
                    include_lowest=True,
                    retbins=True)

# Obtenim un vector amb dues columes, la primera amb le dades categoritzades
# i el segon amb un els rangs dels contenidors
print("Dades categoritzades:  ",resultat[0],  "\n")
print("Contenidors:  ",resultat[1],  "\n")

#  Códigoss  associats
print("  Codis  associats")
print(resultat[0].codes,  "\n")
#  Categorias
print(  "  Categories  creades")
print(resultat[0].categories,  "\n")

#  Mostrem les dades convertides dins un array de numpy 
print("  Datos  convertidos")
print(np.array(resultat[0]))

Dades categoritzades:   ['xiquet', 'xiquet', 'xiquet', 'jove', 'jove', ..., 'adult', 'adult', 'major', 'major', 'major']
Length: 12
Categories (4, object): ['xiquet' < 'jove' < 'adult' < 'major'] 

Contenidors:   [ 0. 11. 17. 59. inf] 

  Codis  associats
[0 0 0 1 1 2 2 2 2 3 3 3] 

  Categories  creades
Index(['xiquet', 'jove', 'adult', 'major'], dtype='object') 

  Datos  convertidos
['xiquet' 'xiquet' 'xiquet' 'jove' 'jove' 'adult' 'adult' 'adult' 'adult'
 'major' 'major' 'major']


In [197]:
# Per saber el nombre d'elements en cada rang podem afegir retbins=false

resultat2  =  pd.cut(edats, bins=[0,  11,  17,  59,  np.inf], labels=["xiquet",  "jove",  "adult",  "major"], include_lowest=True, retbins=True)
pd.Series.value_counts(pd.Series(resultat2)) # value_counts vol una sèrie

[xiquet, xiquet, xiquet, jove, jove, adult, adult, adult, adult, major, major, major]    1
[0.0, 11.0, 17.0, 59.0, inf]                                                             1
Name: count, dtype: int64